# Assignment 4, Aayush Verma,30/07/2024

## Question 2

### Transformer modules are defined in part 1

In [2]:
from assignment_4_aayush_verma_tensorflowTransformer_part1.ipynb import Transformer
import tensorflow as tf
import tensorflow_text as tftext
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'assignment_4_aayush_verma_tensorflowTransformer_part1'

In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
trf = Transformer(num_layers = 6,d_model = 128,num_heads = 8,dff = 512,input_vocab_size = 7765,target_vocab_size = 7010,dropout_rate = 0.1)

In [4]:
en = tf.constant([[1.0,2.0]])
de = tf.constant([[1.0,2.0]])

In [5]:
trf((en,de))

<tf.Tensor: shape=(1, 2, 7010), dtype=float32, numpy=
array([[[-0.2405577 , -0.0151207 ,  0.10619164, ...,  0.07150526,
          0.09766266,  0.21731229],
        [ 0.25176635, -0.36934775,  0.58713216, ..., -0.08684311,
          0.17783323, -0.25247622]]], dtype=float32)>

In [6]:
trf.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  4969088   
                                                                 
 decoder (Decoder)           multiple                  8038912   
                                                                 
 dense_24 (Dense)            multiple                  904290    
                                                                 
Total params: 13912290 (53.07 MB)
Trainable params: 13912290 (53.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps
    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [8]:
learning_rate = CustomSchedule(768)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [9]:
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
    loss = loss_object(label, pred)
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss
def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred
    mask = label != 0
    match = match & mask
    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [10]:
trf.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [11]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en',
                               with_info=True,
                               as_supervised=True)

train_examples, val_examples = examples['train'], examples['validation']

/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [12]:
train_examples, val_examples = examples['train'], examples['validation']

In [13]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

'./ted_hrlr_translate_pt_en_converter.zip'

In [14]:
tokenizers = tf.saved_model.load(model_name)

In [15]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [16]:
MAX_TOKENS=128
def prepare_batch(pt, en):
    pt = tokenizers.pt.tokenize(pt)      # Output is ragged.
    pt = pt[:, :MAX_TOKENS]    # Trim to MAX_TOKENS.
    pt = pt.to_tensor()  # Convert to 0-padded dense Tensor
    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)]
    en_inputs = en[:, :-1].to_tensor()  # Drop the [END] tokens
    en_labels = en[:, 1:].to_tensor()   # Drop the [START] tokens
    return (pt, en_inputs), en_labels

In [17]:
BUFFER_SIZE = 20000
BATCH_SIZE = 16

In [18]:
def make_batches(ds):
    return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

In [19]:
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [20]:
for (pt, en), en_labels in train_batches.take(1):
    break

print(pt.shape)
print(en.shape)
print(en_labels.shape)

(16, 79)
(16, 70)
(16, 70)


In [ ]:
trf.fit(train_batches,
                epochs=20,
                validation_data=val_batches)

Epoch 1/20
3237/3237 [==============================] - 3413s 1s/step - loss: 5.6172 - masked_accuracy: 0.2058 - val_loss: 4.4684 - val_masked_accuracy: 0.3062
Epoch 2/20
3237/3237 [==============================] - 3256s 1s/step - loss: 4.2013 - masked_accuracy: 0.3402 - val_loss: 3.7827 - val_masked_accuracy: 0.3902
Epoch 3/20
3237/3237 [==============================] - 3253s 1s/step - loss: 3.6306 - masked_accuracy: 0.4124 - val_loss: 3.3020 - val_masked_accuracy: 0.4627
Epoch 4/20
3237/3237 [==============================] - 3253s 1s/step - loss: 3.2747 - masked_accuracy: 0.4581 - val_loss: 3.0356 - val_masked_accuracy: 0.5017
Epoch 5/20
3237/3237 [==============================] - 3261s 1s/step - loss: 3.0257 - masked_accuracy: 0.4907 - val_loss: 2.8746 - val_masked_accuracy: 0.5215
Epoch 6/20
3237/3237 [==============================] - 3257s 1s/step - loss: 2.8403 - masked_accuracy: 0.5151 - val_loss: 2.7467 - val_masked_accuracy: 0.5399
Epoch 7/20
3237/3237 [==================

KeyboardInterrupt: 

In [25]:
trf.save_weights('./checkpoint-1/checkpoint-1')

In [27]:
trf.load_weights('./checkpoint-1/checkpoint-1')

In [29]:
trf((tf.constant([[1.0,2.0]]),tf.constant([[1.0,2.0]])))

<tf.Tensor: shape=(1, 2, 7010), dtype=float32, numpy=
array([[[-11.9100895, -11.898217 , -11.903269 , ..., -11.901754 ,
         -11.909857 ,  -9.540294 ],
        [-11.932209 , -11.920357 , -11.925403 , ..., -11.923866 ,
         -11.931978 ,  -9.557922 ]]], dtype=float32)>